# 🧹 TF-IDF 고도화 (Stopword 적용 + 키워드 필터링)

In [ ]:

# 필요 시 설치
# !pip install pandas scikit-learn


In [ ]:

import pandas as pd

df = pd.read_csv("36000_reviews_label.csv")
df = df[['sentence', 'label']].dropna()
df = df[df['label'].isin(['긍정', '부정', '중립'])]


In [ ]:

default_stopwords = [
    '있다', '없다', '하다', '되다', '이다', '같다', '에서', '그리고', '그냥', '정도', '더', '또', '많이', 
    '진짜', '너무', '좀', '아주', '이런', '저런', '그런', '것', '수', '잘', '게', '고', '는', '을', '를', '이', '가'
]


In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer

# stopword 없이
vec_no_stop = TfidfVectorizer(max_features=5000, min_df=5)
vec_no_stop.fit(df['review_content'])
print("불용어 제거 전 단어 수:", len(vec_no_stop.get_feature_names_out()))

# stopword 적용
vec_with_stop = TfidfVectorizer(max_features=5000, min_df=5, stop_words=default_stopwords)
vec_with_stop.fit(df['review_content'])
print("불용어 제거 후 단어 수:", len(vec_with_stop.get_feature_names_out()))


In [ ]:

def get_top_words_by_sentiment(df, label, stopwords=[], top_n=20):
    from sklearn.feature_extraction.text import TfidfVectorizer
    text = df[df['sentiment_tfidf'] == label]['review_content'].dropna().tolist()
    vec = TfidfVectorizer(max_features=3000, stop_words=stopwords)
    X = vec.fit_transform(text)
    mean_tfidf = X.mean(axis=0).A1
    words = vec.get_feature_names_out()
    word_score_pairs = list(zip(words, mean_tfidf))
    return sorted(word_score_pairs, key=lambda x: x[1], reverse=True)[:top_n]

top_pos = get_top_words_by_sentiment(df, '긍정', default_stopwords)
top_neg = get_top_words_by_sentiment(df, '부정', default_stopwords)

print("👍 긍정 키워드:")
for word, score in top_pos: print(f"{word}: {score:.4f}")

print("\n👎 부정 키워드:")
for word, score in top_neg: print(f"{word}: {score:.4f}")
